In [ ]:
# 

In [ ]:
def CreateSimilarlyList(moveList, partList):
    result = []
    for move, part in zip(moveList, partList):
        if move == part == 1:
            result.append(1)
        else: result.append(0)
    return result

def CompareSimilarlyListWithHandmade(similaryList, handmadeList):
    result = []
    for similarly, handmade in zip(similaryList, handmadeList):
        if similarly == handmade:
            if similarly == 1:
                result.append(3)
            else: result.append(2)
        else:
            if handmade == 1:
                result.append(1)
            else: result.append(0)
    return result


# README

## パートと数値の関係
0: None
1: Drum
2: Melody
3: Vocal

## 目視データと出力データの比較の際の数値
0: 出力データのみ
1: 目視データのみ重複タイミング発生
2: 両方重複タイミング無し
3: 両方重複隊ミンングあり